In [ ]:
# !wget https://dropbox.com/s/fnpq3z4bcnoktiv/positive.csv - раскоментируйте, чтобы скачать

In [ ]:
# !wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)



In [3]:
positive.shape

(114911, 2)

In [4]:
negative.shape

(111923, 2)

In [24]:
df.sample(10)

,text,label
48621,"Орала на нас, маркер летал:) а потом сама с с...",positive
82667,"бляяя тупое московское время, в 23 часа по мос...",negative
105656,"поймала себя на мысли, что уже сто лет не пила...",positive
40531,"Песня: битард, битард битард, кам ту май будуа...",negative
87615,@eliseewaalena_ ну если и нет то только потом...,positive
52991,А хочется как в детстве - в канун Рождества ва...,negative
46915,"то самое чувство, когда ты хочешь напиться, но...",negative
99699,"@Volinka_ зато ты будешь отдыхать, а я пахать ...",negative
75384,"чо,испугался? я вот чот тож(((( — чего испугал...",negative
102681,@Snapy_ekb @t_darlin @ellacanary67 @FantinaFan...,positive


In [7]:
df.shape

(226834, 2)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams

In [10]:
sent = 'Если б мне платили каждый раз'.split()
list(ngrams(sent, 1))

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [12]:
list(ngrams(sent, 3))

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

In [14]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)

In [13]:
x_train.shape

(170125,)

In [15]:
bow.shape

(170125, 243557)

In [16]:
list(vec.vocabulary_.items())[:10]

[('rt', 74422),
 ('lenitsky', 50683),
 ('куда', 150091),
 ('еще', 130548),
 ('больше', 107204),
 ('гадким', 118522),
 ('быть', 109196),
 ('http', 37991),
 ('co', 21503),
 ('r607ans2ga', 72071)]

In [17]:
clf = LogisticRegression(random_state=42, max_iter=10000)
clf.fit(bow, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.77      0.76      0.76     28211
    positive       0.76      0.77      0.77     28498

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



In [21]:
vec = CountVectorizer(ngram_range=(2, 2))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=10000)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.69      0.72      0.70     27023
    positive       0.73      0.71      0.72     29686

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



In [22]:
bow.shape

(170125, 1003037)

### TF-IDF

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [37]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=10000)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.73      0.77      0.75     26795
    positive       0.78      0.75      0.77     29914

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



In [27]:
bow.shape

(170125, 243557)

In [28]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marimitchurina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
example = 'Это был плохой опыт:('
word_tokenize(example)

['Это', 'был', 'плохой', 'опыт', ':', '(']

In [ ]:
from nltk import collocations
nltk.download('genesis')

print(type(nltk.corpus.genesis.words('english-web.txt')))
bigram_measures = collocations.BigramAssocMeasures()
bigram_finder = collocations.BigramCollocationFinder.from_documents([nltk.word_tokenize(x) for x in x_train])
bigrams = bigram_finder.nbest(bigram_measures.pmi, 100)
# print(bigrams)

In [ ]:
bigrams = bigram_finder.nbest(bigram_measures.likelihood_ratio, 100)
bigrams

### Стоп-слова и пунктуация

In [30]:
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marimitchurina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
noise = stopwords.words('russian') + list(punctuation)

In [34]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28004
    positive       1.00      1.00      1.00     28705

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=10000)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

In [38]:
max_index = list(clf.coef_[0]).index(max(clf.coef_[0]))
for v in list(vec.vocabulary_.items()):
    if v[1] == max_index:
        print(v[0])
        break

dd


In [40]:
maxs = sorted(list(clf.coef_[0]))

for i in maxs[:30]:
    max_index = list(clf.coef_[0]).index(i)
    for v in list(vec.vocabulary_.items()):
        if v[1] == max_index:
            print(v[0])
            break

o_o
о_о
обидно
cio_optimal
жаль
to_over_kill
скучаю
печально
грустно
99
do_or_die_xxx
жалко
prisonero_o
сожалению
печаль
блин
ужасно
боль
не
скучать
ненавижу
жизньболь
ужас
reno_oppa
увы
kota_oo_oo
грустный
устала
заболела
horanso_on


In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=mystem.lemmatize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))